In [ ]:
import pyodbc
import json
import datetime
import teradata
import pandas as pd
from azure.cosmos import cosmos_client
import urllib3

#logging.captureWarnings(True)
urllib3.disable_warnings()

#Configuration to connect to the container in Azure CosmosDB
CONFIG = 
{"ENDPOINT": "https://<Insert your endopint here>/",
"PRIMARYKEY": "<Insert your primarykey here>==",}

#Cosmos Client creation =CosmosClient(url,credential=key),
client = cosmos_client.CosmosClient(CONFIG["ENDPOINT"], {"masterKey": CONFIG["PRIMARYKEY"]},connection_verify=False)

#Using the client configuration to connect to the database and container
database_name = '<Insert your database name here>'
database = client.get_database_client(database_name)
container_name = '<Insert your container name here>'
container = database.get_container_client(container_name)

#Creation of a dataframe with data from the container
dflist = []
for item in container.query_items(
    query='<Insert your select query here>' ,
    enable_cross_partition_query=True):
    dflist.append(dict(item))
df = pd.DataFrame(dflist)

#Transforming the date columns and reindexing (This step was necessary for my use case)
#This line converts the string object in Timestamp object
df['dateTime'] = [datetime.datetime.strptime(d, "%Y-%m-%d %H:%M:%S") for d in df["dateTime"]]

#extracting date from timestamp
df['Date'] = [datetime.datetime.date(d) for d in df['dateTime']] 

#extracting time from timestamp
df['Time'] = [datetime.datetime.time(d) for d in df['dateTime']] 

#reindexing the columns
df = df.reindex(columns=['<Column 1>','<Column 2>','<Column 3>','<Column 4>','<Column 5>'])

#Tuple creation from the dataframe
data = [tuple(x) for x in df.to_records(index=False)]


#Configuration the connection to Teradata
host,username,password = '<Insert your host name here>','<Insert your username here>', '<Insert your passowrd here>'
#Make a connection
udaExec = teradata.UdaExec (appName="test", version="1.0", logConsole=False)
cnxn = udaExec.connect(method="odbc",system=host, username=username,password=password, 
                         driver="Teradata Database ODBC Driver 16.20") # <--Indicate our ODBC Driver

#Assigning the connection to a cursor
cursor = cnxn.cursor()

#Deleting the previous data in the Teradata intemediate table
try: 
    query0='<Insert your delete query here>'
    cursor.execute(query0)
except Error as e:
    print("Error deleting data",e)
finally:
    cnxn.commit()

#Inserting the tuple data in the Teradata intermediate table
try:
    query1='<Insert your insert query here>'
    cursor.executemany(query1,data,batch=True)
except Error as e:
    print("Error inserting data",e)
finally:
    cnxn.commit()


#Dencrypting the columns from the Teradata intermediate table
try:
    query2= '<Insert your dencryp query here>'
    cursor.execute(query2)
    #Returns a list of tuples with data from the previous cursor execution
    records = cursor.fetchall() 
except Error as e:
    print ("Error dencrypting data",e)
finally:
    cnxn.commit()

#Transforming the list into a final dataframe
dff = pd.DataFrame.from_records(records, columns=['<Column 1>', '<Column 2>', '<Column 3>', '<Column 4>', '<Column 5>'])

#Tuple creation from the final dataframe data
dataf = [tuple(x) for x in dff.to_records(index=False)]

#Deleting the previous data from the Teradata final table
try:
    query3= '<Insert your delete query here>'
    cursor.execute(query3)
except Error as e:
    print ("Error inserting data",e)
finally:
    cnxn.commit()

#Inserting the tuple data in the Teradata final table
try:
    query4= '<Insert your insert query here>'
    cursor.executemany(query4,dataf,batch=True)
except Error as e:
    print ("Error inserting data",e)
finally:
    cnxn.commit()
    cnxn.close()#Closing the connection